In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import pyCloudy as pc

In [ ]:
!mkdir ../SIGNALS_models
dir_ = '../SIGNALS_models'

In [ ]:
model_name = 'M1'
# An object is created (instatiated). It is in charge of the Cloudy input file. 
# Give the name of the model when instantiate the Class to the Min object.
Min = pc.CloudyInput('{}/{}'.format(dir_, model_name))

In [ ]:
help(pc.CloudyInput)

In [ ]:
# Some parameters of the models are transimted to the Min object.
# The ionizing SED is a Blackbody in this example. Other SEDs can also be defined, see set_star method.
Min.set_BB(Teff=40000, lumi_unit='luminosity solar total', lumi_value=5)
# A constant density is used.
Min.set_cste_density(2)
# The inner radius is given (a second argument can set the outer radius)
Min.set_radius(18.5)
# Cloudy predefined abundance set is asumed. Element by element abundance can also be defined, using a dictionnary.
Min.set_abund(predef='ism', nograins=False)
# Anything that is not available by dedicated method can always be done using the "set_other" method.
Min.set_other(('Cosmic Rays Background'))
#Min.set_other(('set dr 0'))
# Stoping criteria can be defined.
#Min.set_stop(('zone = 1'))

In [ ]:
# Once all the model parameters have been defined suing the collection of "set_" methods, the input file is printed out.
Min.print_input()

In [ ]:
!cat ../SIGNALS_models/M1.in

In [ ]:
# The list of the saved file can be edited.
pc.config.SAVE_LIST

In [ ]:
# We tell pyCloudy where the executable is:
#pc.config.cloudy_exe = '/usr/local/Cloudy/c17.02/source/cloudy.exe'
pc.config.cloudy_exe = 'cloudy.exe'

In [ ]:
# We can run Cloudy from within the script:
Min.run_cloudy()

In [ ]:
!ls -l ../SIGNALS_models/M1.*

In [ ]:
# The output files for the M1 model are read by the CloudyModel object when instantiating it.
M = pc.CloudyModel('{}/{}'.format(dir_, model_name), read_emis=False)

In [ ]:
help(pc.CloudyModel)

In [ ]:
# The M object includes a lot of methods to deal with the data saved in the Cloudy model output files.
# A method can give a summary of the model, to check that the results are close to what expected.
M.print_stats()

In [ ]:
# The abundances used in the model are stored into a dictionary.
print(M.abund)

In [ ]:
# Part of the main Cloudy model output file are extracted from the file and stored into a dictionary.
M.out

In [ ]:
# Comments, cautions and warnings are also available.
print(M.comments)
print('-----------------')
print(M.cautions)
print('-----------------')
print(M.warnings)

In [ ]:
# The mean of the ionization parameter over the volume of the nebula is available. 
# It may be weighted by the electron*hydrogen density (useful if the model goes into the neutral region).
print(M.log_U_mean, M.log_U_mean_ne)

In [ ]:
M.log_U_mean_ne?

In [ ]:
M.log_U_mean_ne??

In [ ]:
print(M.vol_mean(M.log_U), np.log10(M.vol_mean(10**M.log_U)))

In [ ]:
# Most of the radial varying parameters are easily accessed and ploted.
f, ax = plt.subplots(figsize=(6,6))
ax.plot(M.radius, M.te);

In [ ]:
# Ionic fraction of any element are accessible to be ploted versus the radius of the nebula.
f, ax = plt.subplots(figsize=(6,6))
ax.plot(M.radius, M.get_ionic('O',4), label=r'O$^{4+}$/O')
ax.plot(M.radius, M.get_ionic('O',3), label=r'O$^{3+}$/O')
ax.plot(M.radius, M.get_ionic('O',2), label=r'O$^{2+}$/O')
ax.plot(M.radius, M.get_ionic('O',1), label=r'O$^{+}$/O')
ax.plot(M.radius, M.get_ionic('O',0), label=r'O$^{0}$/O')
ax.legend(loc='best');

In [ ]:
# Electron temeprature weighted by ionic fraction and electron density are also available.
print(r'<Te>$_{O^0/O.Ne}$ =', M.get_T0_ion_vol_ne('O',0))
print(r'<Te>$_{O^+/O.Ne}$ =', M.get_T0_ion_vol_ne('O',1))
print(r'<Te>$_{O^{2+}/O.Ne}$ =', M.get_T0_ion_vol_ne('O',2))
print(r'<Te>$_{O^{3+}/O.Ne}$ =', M.get_T0_ion_vol_ne('O',3))

In [ ]:
# Ionic fractions (weighted by Ne or not)
print(M.get_ab_ion_vol('O',2))
print(M.get_ab_ion_vol_ne('O',2))

In [ ]:
# A ploting tool to visualize the spectra is available.
help(M.get_cont_x)
print('---------------')
help(M.get_cont_y)
print('---------------')
help(M.plot_spectrum)

In [ ]:
# The area under the red curve and under the blue curve are the same (in IR the blue curve is slightly over the red one)
f, ax = plt.subplots(figsize=(14,10))
M.plot_spectrum(cont='incid', ax=ax, c='r', xunit='Ang', yunit='esA', xlog=False, ylog=False)
M.plot_spectrum(cont='ntrans', ax=ax, xunit='Ang', yunit='esA', xlog=False, ylog=False)
ax.set_xlim((0, 1e5))

In [ ]:
f, ax = plt.subplots(figsize=(14,10))
M.plot_spectrum(ax=ax, xunit='Ang', yunit='esA')
M.plot_spectrum(cont='incid', ax=ax, c='r', xunit='Ang', yunit='esA')
ax.set_xlim((1, 1e7))
ax.set_ylim((1e25, 1e37));

In [ ]:
print(np.log10(M.Q0))

# Grid of models

In [ ]:
# A function is used to produce input file dpeending on a given parameter, here the inner radius.
def make_model(dir_, name, radius):
    # The name of the model must depend on the varying parameter, to avoid overwriting of the files
    Min = pc.CloudyInput('{}/{}_{}'.format(dir_, name, radius)) 
    Min.set_BB(Teff=40000, lumi_unit='Q(H)', lumi_value=48.7)
    Min.set_cste_density(2)
    Min.set_radius(radius)
    Min.set_abund(predef='ism', nograins=False)
    Min.set_other(('Cosmic Rays Background'))
    #Min.set_other(('set dr 0'))
    #Min.set_stop(('zone = 1'))    
    Min.print_input() #We do not run the model, only print the input file

In [ ]:
# A grid of 6 model input files is done.
name = 'M2'
for radius in np.linspace(13, 23,6):
    make_model(dir_, name, radius)

In [ ]:
# This will create a Makefile in the model directory
# The make command is used to run the models in parallel
pc.print_make_file(dir_)

In [ ]:
!cat ../SIGNALS_models/Makefile

In [ ]:
# The 6 models are run using the Makefile
pc.run_cloudy(dir_=dir_, n_proc=6, use_make=True)

In [ ]:
# The models are read into a list of CloudyModel objects
Ms = pc.load_models('{}/{}'.format(dir_, name), read_emis=False)

In [ ]:
# The list is sorted according to the inner radius
Ms = sorted(Ms, key = lambda M:M.r_in)

In [ ]:
# It is very easy to loop on any of the model parameters. Here we explore the string describing how Cloudy ended.
for M in Ms:
    print(M.model_name_s, M.out['Cloudy ends'])

In [ ]:
# Using list comprehension, it is easy to generate tables of the varying parameters.
print('Inner R', ['{:8.2f}'.format(np.log10(M.r_in)) for M in Ms])
print('Outer R', ['{:8.2f}'.format(np.log10(M.r_out)) for M in Ms])
print('<Te>   ', ['{:8.2f}'.format(M.T0) for M in Ms])
print('<logU> ', ['{:8.2f}'.format(M.log_U_mean_ne) for M in Ms])
print('H0/H   ', ['{:8.2f}'.format(M.get_ab_ion_vol_ne('H',0)) for M in Ms])
print('H+/H   ', ['{:8.2f}'.format(M.get_ab_ion_vol_ne('H',1)) for M in Ms])
print('H2/H   ', ['{:8.2f}'.format(2*M.get_ab_ion_vol_ne('H',2)) for M in Ms])
print('O++/O  ', ['{:8.2f}'.format(M.get_ab_ion_vol_ne('O',2)) for M in Ms])

In [ ]:
# Overplotting of the spectra of all the models is easy obtained
f, ax = plt.subplots(figsize=(14, 10))
M.plot_spectrum(label='INPUT', ax=ax, xunit='Ang', cont='incid')
for M in Ms:
    M.plot_spectrum(label=M.model_name_s, ax=ax, xunit='Ang')
ax.legend()
ax.set_xlim((10, 1e8))